### Урок 2. Домашка

### Домашнее задание

1. теория
2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)
3. Повторить п.2, но используя уже не медиану, а max
4. (опциональное, если очень хочется) не стал делать
5. Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score
6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

### Подключение библиотек. Загрузка датасетов (повторяем, что было на уроке)

Три ячейки ниже делаем в Google Colaboratory

- установим razdel
- установим pymorphy2

In [13]:
!pip install razdel

In [12]:
!pip install pymorphy2

     |████████████████████████████████| 61kB 2.4MB/s 
     |████████████████████████████████| 8.2MB 7.5MB/s 


Делаем доступ к Googl Drive

In [4]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [5]:
import pandas as pd

Наши новости

In [6]:
news = pd.read_csv("/gdrive/MyDrive/Colab Notebooks/GB/gb4sm-ml-in-business/articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


Загрузим пользователей и списки последних прочитанных новостей

In [7]:
users = pd.read_csv("/gdrive/MyDrive/Colab Notebooks/GB/gb4sm-ml-in-business/users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


Итак, нам нужно получить векторные представления пользователей на основе прочитанным ими новостей и самих новостей

### 1. Получаем векторные представления новостей (повторяем, что было на уроке)

In [10]:
#from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

In [14]:
#предобработка текстов
import re
import numpy as np
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel

import pymorphy2  # pip install pymorphy2

In [15]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
stopword_ru = stopwords.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

In [17]:
with open('/gdrive/MyDrive/Colab Notebooks/GB/gb4sm-ml-in-business/stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [20]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [21]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

CPU times: user 19.9 s, sys: 561 ms, total: 20.4 s
Wall time: 20.5 s


In [22]:
news.head(10)

,doc_id,title
0,6,заместитель председателяnправительства рфnсерг...
1,4896,матч финала кубка россии по футболу был приос...
2,4897,форвард авангарда томаш заборский прокомментир...
3,4898,главный тренер кубани юрий красножан прокоммен...
4,4899,решением попечительского совета владивостокско...
5,4900,ио главного тренера вячеслав буцаев прокоммент...
6,4901,запорожский металлург дома потерпел разгромное...
7,4902,сборная сша одержала победу над австрией со сч...
8,4903,бывший защитник сборной россии дарюс каспарайт...
9,4904,полузащитник цска зоран тошич после победы над...


In [23]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

CPU times: user 3min 53s, sys: 266 ms, total: 3min 54s
Wall time: 3min 54s


In [24]:
news.head(10)

,doc_id,title
0,6,"[заместитель, председатель, правительство, рф,..."
1,4896,"[матч, финал, кубок, россия, футбол, приостано..."
2,4897,"[форвард, авангард, томаш, заборский, прокомме..."
3,4898,"[главный, тренер, кубань, юрий, красножанин, п..."
4,4899,"[решение, попечительский, совет, владивостокск..."
5,4900,"[ио, главное, тренер, вячеслав, буцай, прокомм..."
6,4901,"[запорожский, металлург, дом, потерпеть, разгр..."
7,4902,"[сборная, сша, одержать, победа, австрия, счёт..."
8,4903,"[бывший, защитник, сборная, россия, дарюс, кас..."
9,4904,"[полузащитник, цска, зоран, тошич, победа, тер..."


А теперь в 3 строчки обучим нашу модель

In [25]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Что такое common_dictionary и как он выглядит

In [26]:
common_dictionary[15]

'генералполковник'

In [27]:
common_corpus[0][0]

(0, 5)

In [28]:
len(texts[0])

118

Все просто - это словарь наших слов

Запускаем обучение

In [29]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

CPU times: user 54 s, sys: 25.9 s, total: 1min 19s
Wall time: 51.2 s


In [30]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [31]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'играть', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'хороший']


[(2, 0.035618287),
 (3, 0.2828906),
 (7, 0.040201318),
 (15, 0.28535202),
 (19, 0.048918486),
 (22, 0.06912254),
 (24, 0.2203358)]

In [32]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: учёный исследователь журнал исследование страдать женщина газета
topic_1: самолёт американский рак двигатель nasa космический древний
topic_2: гражданин мужчина миссия космос ребёнок убийство сотрудник
topic_3: век узнать солнце считаться увидеть nn сайт
topic_4: россия банк сша нефть глава санкция решение
topic_5: украина поверхность киев девочка украинский сон золото
topic_6: погибнуть nn место пассажир авария астероид ветер
topic_7: доллар расчёт фестиваль мкс медведев лунный астронавт
topic_8: ракета проверка планета орбита польша первый польский
topic_9: церковь общество мышь городской московский лётчик статья
topic_10: экономика млн мозг выяснить объём врач дом
topic_11: nn площадь обнаружить высота аэропорт участок выяснить
topic_12: животное налог дыра норма анализ пища освобождение
topic_13: статья операция район пострадать клетка уголовный орган
topic_14: русский законодательство миллиард разместить писать сенатор юрист
topic_15: вода инвестиция первый корабль темпер

Очень неплохо - большинство тем вполне можно описать о чем они


построим темы 

In [33]:
news['title'].iloc[5]

['ио',
 'главное',
 'тренер',
 'вячеслав',
 'буцай',
 'прокомментировать',
 'победа',
 'югра',
 'матч',
 'регулярный',
 'чемпионат',
 'кхл',
 'nn',
 'прежде',
 'хотеть',
 'поблагодарить',
 'ребята',
 'команда',
 'передавать',
 'официальный',
 'сайт',
 'югра',
 'игра',
 'начинаться',
 'очень',
 'удачно',
 'пропустить',
 'гол',
 'первый',
 'смена',
 'долго',
 'входить',
 'игра',
 'связать',
 'дневный',
 'матч',
 'непривычный',
 'плюс',
 'третий',
 'матч',
 'сибирь',
 'гдеть',
 'ребята',
 'чувствовать',
 'скажем',
 'сонный',
 'второй',
 'третий',
 'период',
 'показать',
 'разбежаться',
 'играть',
 'правильно',
 'появиться',
 'момент',
 'дотерпеть',
 'заслуженно',
 'победить']

In [34]:
lda[common_dictionary.doc2bow(news['title'].iloc[5])]

[(3, 0.34382766), (7, 0.03088229), (13, 0.040857263), (24, 0.57019544)]

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [35]:
#text = news['title'].iloc[0]

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [36]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.0,0.0,0.0,0.000000,0.453072,0.000000,0.0,0.000000,0.09282,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.125549,0.00000,0.0,0.000000,0.0,0.3215,0.0,0.0,0.000000
1,4896,0.0,0.0,0.0,0.300313,0.000000,0.000000,0.0,0.035037,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.643187,0.0,0.0000,0.0,0.0,0.000000
2,4897,0.0,0.0,0.0,0.280425,0.000000,0.000000,0.0,0.075287,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.278753,0.000000,0.08628,0.0,0.044492,0.0,0.0000,0.0,0.0,0.216227
3,4898,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.148177,0.000000,0.00000,0.0,0.089474,0.0,0.0000,0.0,0.0,0.751872
4,4899,0.0,0.0,0.0,0.064720,0.000000,0.031586,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.529998,0.00000,0.0,0.350994,0.0,0.0000,0.0,0.0,0.000000


In [37]:
topic_matrix.shape

(27000, 26)

Прекрасно, мы получили вектора наших новостей! И даже умеем интерпретировать получившиеся темы.

Можно двигаться далее

### Следующий шаг - векторные представления пользователей (здесь уже начинается домашка)

In [38]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [40]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [41]:
doc_dict[293622]

array([0.03673285, 0.12548508, 0.        , 0.30841601, 0.1162037 ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.03353786, 0.        , 0.        , 0.0205492 , 0.        ,
       0.        , 0.11866403, 0.11833912, 0.        , 0.        ,
       0.04115508, 0.        , 0.        , 0.07446545, 0.        ])

In [42]:
user_articles_list = users['articles'].iloc[33]

def get_user_embedding(user_articles_list, func):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = func(user_vector, 0)
    return user_vector

In [52]:
get_user_embedding(user_articles_list, np.mean)

array([0.        , 0.00680746, 0.07380305, 0.        , 0.17488927,
       0.00175764, 0.        , 0.0032858 , 0.01526561, 0.00475175,
       0.00678313, 0.04571896, 0.        , 0.        , 0.00530302,
       0.0210873 , 0.09400701, 0.13073614, 0.15388632, 0.03854015,
       0.00559196, 0.00726437, 0.03699381, 0.12633852, 0.0370222 ])

Интересовался новостями с топиками topic_3, topic_14 (что-то про политику и государство)

Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

In [103]:
result = []
for func in [np.mean, np.median, np.max]:
  item = {'func': func, 
          'user_embendings': None, 
          'X': None, 
          'X_train': None,
          'X_test': None,
          'y_train': None,
          'y_test': None,
          'logreg': None,
          'preds': None,
          'precision': None,
          'recall': None,
          'thresholds': None,
          'fscore': None,
          'roc_auc': None
          }
  result.append(item)

In [104]:
for item in result:
  user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, item['func']), 1)])
  user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
  user_embeddings['uid'] = users['uid'].values
  user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]

  item['user_embendings'] = user_embeddings


#user_embeddings.head(3)  

In [93]:
#посмотрим user_embeddings для mean
result[0]['user_embendings'].head()


,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.006122,0.029419,0.014561,0.147488,0.073041,0.052246,0.001972,0.000000,0.006811,0.000000,0.005590,0.052766,0.000000,0.033050,0.042678,0.001880,0.116258,0.065428,0.006549,0.080632,0.014009,0.019953,0.027557,0.185921,0.000000
1,u108690,0.007766,0.013895,0.071250,0.005967,0.219925,0.044756,0.003557,0.005692,0.000000,0.009086,0.028863,0.041488,0.024617,0.037055,0.016859,0.003094,0.063316,0.031703,0.012078,0.062273,0.000000,0.026788,0.064647,0.090410,0.097231
2,u108339,0.004355,0.000000,0.079069,0.007889,0.162788,0.000000,0.038221,0.002222,0.000000,0.000000,0.020312,0.095157,0.001775,0.080753,0.018454,0.002454,0.070053,0.032363,0.066488,0.053798,0.003533,0.086010,0.030335,0.097504,0.026290
3,u101138,0.013543,0.006772,0.063838,0.056046,0.000000,0.090550,0.034433,0.015655,0.026651,0.004505,0.005435,0.000000,0.000000,0.006329,0.003505,0.127390,0.000000,0.052624,0.026335,0.075980,0.068630,0.012491,0.025717,0.031021,0.232385
4,u108248,0.033381,0.000000,0.039123,0.037102,0.116434,0.020237,0.069633,0.004768,0.003398,0.022906,0.068839,0.020506,0.037342,0.025296,0.011900,0.028844,0.084765,0.017328,0.020374,0.068951,0.001821,0.022319,0.082913,0.092210,0.049074


In [67]:
#посмотрим user_embeddings для max
result[2]['user_embendings'].head()

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.036733,0.125485,0.087369,0.385163,0.278905,0.313474,0.011831,0.000000,0.040865,0.000000,0.033538,0.183148,0.000000,0.177751,0.175176,0.011280,0.366539,0.191868,0.039291,0.172273,0.042896,0.091514,0.098864,0.485920,0.000000
1,u108690,0.046595,0.083369,0.269954,0.019870,0.582368,0.140508,0.011121,0.020369,0.000000,0.054518,0.090467,0.118871,0.069962,0.183747,0.065994,0.018562,0.155343,0.089899,0.052465,0.141601,0.000000,0.126791,0.156112,0.281060,0.251861
2,u108339,0.015997,0.000000,0.257945,0.030760,0.460235,0.000000,0.115602,0.013334,0.000000,0.000000,0.054469,0.253990,0.010652,0.183673,0.110723,0.014727,0.335691,0.076207,0.183539,0.125971,0.021198,0.240201,0.099147,0.217371,0.064077
3,u101138,0.058353,0.040631,0.233474,0.238508,0.000000,0.291734,0.206601,0.048400,0.159909,0.013755,0.032610,0.000000,0.000000,0.020423,0.021031,0.529036,0.000000,0.165502,0.158011,0.370511,0.242179,0.074949,0.080439,0.109833,0.456834
4,u108248,0.111069,0.000000,0.094461,0.201825,0.385154,0.121419,0.208255,0.028609,0.020391,0.118599,0.298461,0.080747,0.139226,0.079130,0.052699,0.153136,0.248823,0.072314,0.065594,0.169516,0.010926,0.062408,0.161828,0.310063,0.164888


Датасет готов - можно попробовать обучить модель. Загрузим нашу разметку

In [68]:
target = pd.read_csv('/gdrive/MyDrive/Colab Notebooks/GB/gb4sm-ml-in-business/users_churn.csv')
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [105]:
for item in result:
  item['X'] = pd.merge(item['user_embendings'], target, 'left')

#посмотрим пример для max
result[2]['X'].head()

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,churn
0,u105138,0.036733,0.125485,0.087369,0.385163,0.278905,0.313474,0.011831,0.000000,0.040865,0.000000,0.033538,0.183148,0.000000,0.177751,0.175176,0.011280,0.366539,0.191868,0.039291,0.172273,0.042896,0.091514,0.098864,0.485920,0.000000,0
1,u108690,0.046595,0.083369,0.269954,0.019870,0.582368,0.140508,0.011121,0.020369,0.000000,0.054518,0.090467,0.118871,0.069962,0.183747,0.065994,0.018562,0.155343,0.089899,0.052465,0.141601,0.000000,0.126791,0.156112,0.281060,0.251861,1
2,u108339,0.015997,0.000000,0.257945,0.030760,0.460235,0.000000,0.115602,0.013334,0.000000,0.000000,0.054469,0.253990,0.010652,0.183673,0.110723,0.014727,0.335691,0.076207,0.183539,0.125971,0.021198,0.240201,0.099147,0.217371,0.064077,1
3,u101138,0.058353,0.040631,0.233474,0.238508,0.000000,0.291734,0.206601,0.048400,0.159909,0.013755,0.032610,0.000000,0.000000,0.020423,0.021031,0.529036,0.000000,0.165502,0.158011,0.370511,0.242179,0.074949,0.080439,0.109833,0.456834,0
4,u108248,0.111069,0.000000,0.094461,0.201825,0.385154,0.121419,0.208255,0.028609,0.020391,0.118599,0.298461,0.080747,0.139226,0.079130,0.052699,0.153136,0.248823,0.072314,0.065594,0.169516,0.010926,0.062408,0.161828,0.310063,0.164888,1


In [77]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [106]:
#разделим данные на train/test
for item in result:
  X = item['X']
  item['X_train'], item['X_test'], item['y_train'], item['y_test'] = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)
 
#посмотрим для max
result[2]['X_train'].shape, result[2]['y_train'].shape, result[2]['X_test'].shape, result[2]['y_test'].shape

((6000, 25), (6000,), (2000, 25), (2000,))

In [107]:
for item in result:
  item['logreg'] = LogisticRegression()
  #обучим наш пайплайн
  item['logreg'].fit(item['X_train'], item['y_train'])

In [108]:
for item in result:
  #наши прогнозы для тестовой выборки
  item['preds'] = item['logreg'].predict_proba(item['X_test'])[:, 1]

In [101]:
#посмотрим mean
result[0]['preds'][:10]

array([0.22028335, 0.02466419, 0.64155444, 0.17153499, 0.03162609,
       0.01912997, 0.07368393, 0.06305652, 0.14509531, 0.14293979])

In [99]:
#посмотрим median
result[1]['preds'][:10]

array([0.22304962, 0.0509476 , 0.53456514, 0.13073341, 0.01510098,
       0.02057859, 0.11213357, 0.03698096, 0.11889475, 0.05871333])

In [100]:
#посмотрим median
result[2]['preds'][:10]

array([6.63881441e-02, 7.86636449e-04, 8.70429459e-01, 2.44709184e-01,
       4.33719657e-02, 7.62836593e-02, 4.31836339e-02, 2.50382212e-02,
       6.12817374e-02, 2.76287767e-01])

In [109]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

### Рассчитаем Precision, Recall, F_score

In [114]:
for item in result:
  precision, recall, thresholds = precision_recall_curve(item['y_test'], item['preds'])
  fscore = (2 * precision * recall) / (precision + recall)

  ix = np.argmax(fscore)
  item['precision']   = precision[ix]
  item['recall']      = recall[ix]
  item['thresholds']  = thresholds[ix]
  item['fscore']      = fscore[ix]

  item['roc_auc']      = roc_auc_score(item['y_test'], item['preds'])

## Итоговый результат

In [121]:
data = []
for item in result:
  data.append([item['func'].__name__, item['precision'], item['recall'], item['fscore'], item['roc_auc']])

table_result = pd.DataFrame(data, columns=['func', 'precision', 'recall', 'fscore', 'roc_auc'])
table_result


,func,precision,recall,fscore,roc_auc
0,mean,0.634069,0.820408,0.715302,0.956781
1,median,0.772201,0.816327,0.793651,0.974047
2,amax,0.706320,0.775510,0.739300,0.961286


__Выводы__
С учетом, что мы получили в функции `get_user_embedding` в переменную `user_vector` профиль распредения тем по документам (вероятность соответствия документу определенной теме), то наиболее логичным мне казалось брать именно максимальное значение, т.е. тему с наибольшим значением вероятности. Брать среднее значение вероятности, мне кажется не очень логичным.

Как видим показатели этой функции оказались на втором месте. Но почему именно медиана показала максимальный метрики, я так и не понял.
